# **UMG - Base de Datos 1**

## **Lenguaje de definición de datos - Data definition lenguaje DDL**

Es un lenguaje a nivel de base de datos que permite la creación y/o modificación de objectos como tablas, indices, usuarios.  La sintaxis es similar a la utilizada por un lenguaje programación para definir estructas de datos, esquemas y otros.  Los comandos más comunes son **CREATE**, **ALTER** y **DROP**.

## **<u>Sentencia CREATE</u>**

Esta instrucción es utilizada para crear base de datos, tablas, indices, vistas, procedimientos almacenados y otros.

<u>Ejemplo # 1.1 - Crear base de datos</u>

In [ ]:
CREATE DATABASE [Hospital]

Ejemplo # 1.2 - Crear base de datos con otros parametros

In [ ]:
USE [Master]
GO
CREATE DATABASE [Hospital]
ON PRIMARY
(NAME = Hospital_datafile, FILENAME = 'C:\SQLDATA\Hospital_datafile.mdf', 
SIZE = 100MB, FILEGROWTH = 100MB
)
LOG ON
(NAME = Hospital_transaction, FILENAME = 'D:\SQLlog\Hospital_logfile.ldf', SIZE = 50MB, FILEGROWTH = 50MB)

**Fuente**:  https://docs.microsoft.com/en-us/sql/t-sql/statements/create-database-transact-sql?view=sql-server-ver15

<u>Ejemplo # 2.1 - Crear Tabla</u>

In [ ]:
USE [Hospital] -- utilizacion de base de datos
GO
CREATE TABLE Medico (
    MedicoID INT IDENTITY PRIMARY KEY, 
    Nombre_Medico VARCHAR(150) NOT NULL,
    Apellido_Medico VARCHAR(150) NOT NULL,
    Especialidad VARCHAR(150),
    Fecha_Contratacion DATETIME
)


<u> Ejemplo # 2.2 Crear Tabla utilizando validaciones y valores default </u>

In [ ]:
USE [Hospital]
GO
CREATE TABLE Paciente (
    PacienteID INT IDENTITY PRIMARY KEY, --Llave primaria
    Nombre_Paciente VARCHAR(150) NOT NULL,
    Apellido_Paciente VARCHAR(150) NOT NULL,
    Fecha_Inscripcion DATETIME DEFAULT (GETDATE()), --Valor Default la fecha del sistema
    CHECK (Fecha_Inscripcion > '2020-01-01') --validación de fecha
)

<u> Ejemplo # 2.3 Crear Tabla asociativa y/o debil </u>

In [ ]:
-- 2.3.1
/*USE [Hospital]
GO
CREATE TABLE Paciente_Medico(
    PacMed_ID int IDENTITY,
    MedicoID INT
        REFERENCES dbo.Medico(MedicoID), --REFERENCES 
    PacienteID INT
        REFERENCES dbo.Paciente(PacienteID)        
)*/
--- 2.3.2
USE [Hospital]
GO
CREATE TABLE Paciente_Medico(
    PacMed_ID int IDENTITY(1,1), --(seed,increment)
    MedicoID INT,
    PacienteID INT,
    CONSTRAINT FK_Medico_MedicoID -- validar Integridad Referencial
        FOREIGN KEY (MedicoID)
        REFERENCES dbo.Medico(MedicoID)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
    CONSTRAINT FK_Paciente_PacienteID -- validar Integridad Referencial
        FOREIGN KEY (PacienteID)
        REFERENCES dbo.Paciente(PacienteID)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
    CONSTRAINT PK_PacMed_ID -- validad de Entidad
        PRIMARY KEY CLUSTERED (PacMed_ID)
)

**Fuente**:  https://docs.microsoft.com/en-us/sql/t-sql/statements/create-table-transact-sql?view=sql-server-ver15

<u>Ejemplo # 2.4 Crear indice </u>


In [ ]:
CREATE NONCLUSTERED INDEX IDX_Paciente_Fecha ON dbo.Paciente(Fecha_Inscripcion ASC)

Fuente: https://docs.microsoft.com/en-us/sql/t-sql/statements/create-index-transact-sql?view=sql-server-ver15

<u>Ejemplo # 2.5 Crear Procedimiento Almacenado o Stored Procedure

In [ ]:
CREATE PROCEDURE Reporte_Pacientes @fechai datetime
AS
--Reporte de pacientes inscritos segun la fecha enviada por parametro.
    SET NOCOUNT ON
    SELECT nombre_paciente, apellido_paciente 
    FROM Paciente 
    WHERE fecha_inscripcion >= @fechai
GO

Fuente: https://docs.microsoft.com/en-us/sql/t-sql/statements/create-procedure-transact-sql?view=sql-server-ver15

<u>Ejemplo 2.6 Crear un Login</u>

In [ ]:
CREATE LOGIN md_jperez WITH PASSWORD = '123', CHECK_EXPIRATION =OFF,CHECK_POLICY = OFF,DEFAULT_DATABASE = Hospital
CREATE LOGIN md_faroche WITH PASSWORD = '456', CHECK_EXPIRATION =OFF,CHECK_POLICY = OFF,DEFAULT_DATABASE = Hospital
CREATE LOGIN md_arodas WITH PASSWORD = '789', CHECK_EXPIRATION =OFF,CHECK_POLICY = OFF,DEFAULT_DATABASE = Hospital

<u>Ejemplo 2.7 Crear un usuario a nivel de base de datos y establecer roles de lectura y escritura</u>

In [ ]:
-- CREAR USUARIO A NIVEL DE BASE DE DATOS
USE [Hospital]
CREATE USER md_jperez FOR LOGIN md_jperez
CREATE USER md_faroche FOR LOGIN md_faroche
CREATE USER md_arodas FOR LOGIN md_arodas
-- ASIGNAR PERMISOS DE LECTURA
EXEC sp_addrolemember 'db_datareader','md_jperez'
EXEC sp_addrolemember 'db_datawriter','md_jperez'
-- ASIGNAR PERMISOS DE ESCRITURA
EXEC sp_addrolemember 'db_datareader','md_faroche'
EXEC sp_addrolemember 'db_datawriter','md_faroche'

Fuente: https://docs.microsoft.com/en-us/sql/t-sql/statements/create-login-transact-sql?view=sql-server-ver15

## **<u>Sentencia ALTER</u>**
Instrucción utilizada para la modicación de objetos en SGDB

<u>Ejemplo # 3.1 Modificar Tabla - Agregando Columnas o Campos</u>

In [ ]:
ALTER TABLE Paciente ADD Rango_Crediticio int DEFAULT(1) CHECK (Rango_Crediticio >=1 and Rango_Crediticio <= 3 )

<u>Ejemplo # 3.2 Modificar Tabla - Removiendo Columnas o Campos</u>

In [ ]:
ALTER TABLE Medico DROP COLUMN Especialidad -- NECESARIO COLOCAR 'DROP COLUMN' 

Fuente: https://docs.microsoft.com/en-us/sql/t-sql/statements/alter-table-transact-sql?view=sql-server-ver15

<u>Ejemplo 3.3 Reconstruir un indice</u>


In [ ]:
ALTER INDEX IDX_Paciente_Fecha ON dbo.PACIENTE REBUILD  -- otras opciones -> REORGANIZE | DISABLE

<u>Ejemplo 3.4 Modificar un Stored Procedure </u>


In [ ]:
ALTER PROCEDURE Reporte_Pacientes @fechai datetime
AS
--Reporte de pacientes inscritos segun la fecha enviada por parametro.
    SET NOCOUNT ON
    SELECT nombre_paciente, apellido_paciente, fecha_inscripcion 
    FROM Paciente where fecha_inscripcion >= @fechai
GO

<u>Ejemplo 3.5 Modificar un Login</u>


In [ ]:
ALTER LOGIN md_faroche DISABLE
ALTER LOGIN md_jperez WITH PASSWORD = 'c4Mb10' OLD_PASSWORD = '123'

Fuente: https://docs.microsoft.com/en-us/sql/t-sql/statements/alter-login-transact-sql?view=sql-server-ver15

## **<u>Sentecia DROP</u>**
Instrucción utilizada para la eliminación permanente de objectos en la base de datos

In [ ]:
USE [Hospital]
-- BORRA PROCEDIMIENTO ALMACENADO
DROP PROCEDURE Reporte_Pacientes
DROP INDEX IDX_Paciente_Fecha ON Paciente
DROP TABLE Paciente_Medico
DROP TABLE Paciente

USE [master]
DROP LOGIN [md_arodas]
DROP LOGIN [md_faroche]
DROP LOGIN [md_jperez]
 

## **<u> Sentencias GRANT, DENY y REVOKE</u>**
Estas sentencias son utilizadas para otorgar o remover permisos y/o privilegios sobre objectos de la base de datos. Estos privilegios puede ser SELECT, UPDATE, INSERT, DELETE, EXECUTE, REFERENCES, ALTER,  ALL y OTROS

REVOKE: Comando deprecado en sql server 2019, se mantiene por compatibilidad

<u> Ejemplo </u>

In [ ]:
USE [Hospital]
GRANT SELECT ON dbo.Paciente TO md_arodas -- permisos de SELECT
GRANT EXECUTE ON dbo.Reporte_Pacientes to md_arodas  -- Permisos de Ejecucion
DENY SELECT ON dbo.Medico TO md_faroche -- 
--REVOKE ALL TO md_faroche -- DEPRECADO | Eliminacion de permisos
GRANT ALTER on dbo.Reporte_Pacientes to md_jperez -- permisos de modificacion de objecto 

Fuentes: 

https://docs.microsoft.com/en-us/sql/t-sql/statements/grant-object-permissions-transact-sql?view=sql-server-ver15

https://docs.microsoft.com/en-us/sql/t-sql/statements/deny-transact-sql?view=sql-server-ver15

https://docs.microsoft.com/en-us/sql/t-sql/statements/revoke-transact-sql?view=sql-server-ver15
